In [13]:
from openai import OpenAI
from langchain.utilities import DuckDuckGoSearchAPIWrapper
import yfinance
import json
from dotenv import load_dotenv
import os


In [14]:
load_dotenv()
SERVICE_KEY = os.getenv('OPEN_API_KEY')

def get_ticker(inputs):
    ddg = DuckDuckGoSearchAPIWrapper()
    company_name = inputs["company_name"]
    return ddg.run(f"Ticker symbol of {company_name}")

def get_income_statement(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.income_stmt.to_json())


def get_balance_sheet(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.balance_sheet.to_json())


def get_daily_stock_performance(inputs):
    ticker = inputs["ticker"]
    stock = yfinance.Ticker(ticker)
    return json.dumps(stock.history(period="3mo").to_json())

functions = [
    {
        "type": "function",
        "function": {
            "name": "get_ticker",
            "description": "Given the name of a company returns its ticker symbol",
            "parameters": {
                "type": "object",
                "properties": {
                    "company_name": {
                        "type": "string",
                        "description": "The name of the company",
                    }
                },
                "required": ["company_name"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_income_statement",
            "description": "Given a ticker symbol (i.e AAPL) returns the company's income statement.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_balance_sheet",
            "description": "Given a ticker symbol (i.e AAPL) returns the company's balance sheet.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_daily_stock_performance",
            "description": "Given a ticker symbol (i.e AAPL) returns the performance of the stock for the last 100 days.",
            "parameters": {
                "type": "object",
                "properties": {
                    "ticker": {
                        "type": "string",
                        "description": "Ticker symbol of the company",
                    },
                },
                "required": ["ticker"],
            },
        },
    },
]


In [17]:


client = OpenAI(
    api_key=SERVICE_KEY
)
# assistant = client.beta.assistants.create(
#     name="Investor Assistant",
#     instructions="You help users do research on publicly traded companies and you help users decide if they should buy the stock or not.",
#     tools=functions,
#     model="gpt-4-1106-preview",
# )

ASSISTANT_ID = os.getenv('INVEST_ASSISTANT_ID')

assistant_id = ASSISTANT_ID

In [18]:
thread = client.beta.threads.create(
    messages=[
        {
            "role": "user",
            "content": "I want to know if the Salesforce stock is a good buy",
        }
    ]
)
thread

Thread(id='thread_tYTZ7NjjyZ4GUuJj8ORRJB9W', created_at=1724639009, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))

In [19]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant_id,
)
run

Run(id='run_yeouPi8fzGYSIy9hwUMXPwRN', assistant_id='asst_EGsGfchy18zm3aKOwxJaKnST', cancelled_at=None, completed_at=None, created_at=1724639019, expires_at=1724639619, failed_at=None, incomplete_details=None, instructions='You help users do research on publicly traded companies and you help users decide if they should buy the stock or not.', last_error=None, max_completion_tokens=None, max_prompt_tokens=None, metadata={}, model='gpt-4-1106-preview', object='thread.run', parallel_tool_calls=True, required_action=None, response_format='auto', started_at=None, status='queued', thread_id='thread_tYTZ7NjjyZ4GUuJj8ORRJB9W', tool_choice='auto', tools=[FunctionTool(function=FunctionDefinition(name='get_ticker', description='Given the name of a company returns its ticker symbol', parameters={'type': 'object', 'properties': {'company_name': {'type': 'string', 'description': 'The name of the company'}}, 'required': ['company_name']}, strict=False), type='function'), FunctionTool(function=Functio

In [20]:
def get_run(run_id, thread_id):
    return client.beta.threads.runs.retrieve(
        run_id=run_id,
        thread_id=thread_id,
    )


def get_messages(thread_id):
    messages = client.beta.threads.messages.list(thread_id=thread_id)
    messages = list(messages)
    messages.reverse()
    for message in messages:
        print(f"{message.role}: {message.content[0].text.value}")

In [21]:
get_messages(thread.id)

user: I want to know if the Salesforce stock is a good buy
assistant: To evaluate whether Salesforce stock is a good buy, we should consider several key factors, including:

1. Financial Health: By analyzing the company's financial statements, such as the income statement and the balance sheet, we can assess Salesforce's profitability, revenue growth, expenses, debt levels, and overall financial stability.

2. Stock Performance: Examining the stock's performance over a recent period can give us an idea of market sentiment and how the stock has been trading relative to the market.

3. Market Environment: Considering the current market environment, including competition, economic conditions, and sector performance, is crucial.

4. Valuation Metrics: Looking at valuation ratios such as the price-to-earnings (P/E) ratio, price-to-sales (P/S) ratio, and others help determine if the stock is undervalued or overvalued compared to its peers and historical values.

5. Future Outlook: Understand